### Домашнее задание по лекции "Рекуррентные сети 3" 

1. Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
2. Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
3. Попробуйте добавить +1 рекуррентный в encoder и decoder
4. Попробуйте заменить GRU ячейки на lstm-ячейки
Оцените качество во всех случаях

In [1]:
%matplotlib inline

In [1]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# !wget https://download.pytorch.org/tutorial/data.zip
# !unzip data.zip
# !wget https://www.manythings.org/anki/data.zip
# !unzip /rus-eng.zip

In [2]:
!tail data/eng-rus.txt

"tail" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [3]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())    
    s = re.sub(r'([.!?])', r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)    
    return s
# lines = open('data/%s-%s.txt' % ('eng', 'rus'), encoding='utf-8').read().strip().split('\n')[:10]
# print(lines)
# pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]
# pairs

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 444587 sentence pairs
Trimmed to 25903 sentence pairs
Counting words...
Counted words:
rus 9706
eng 4150
['вы потрясающие .', 'you re amazing .']


The Encoder
-----------





In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------




In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [18]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [179]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    # showPlot(plot_losses)
    

In [20]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [21]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [22]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [180]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 10000, print_every=1000)

1m 35s (- 14m 21s) (1000 10%) 3.5616
3m 14s (- 12m 56s) (2000 20%) 3.3436
4m 52s (- 11m 21s) (3000 30%) 3.2549
6m 32s (- 9m 48s) (4000 40%) 3.1844
8m 9s (- 8m 9s) (5000 50%) 3.0269
9m 48s (- 6m 32s) (6000 60%) 2.9874
11m 28s (- 4m 55s) (7000 70%) 2.9215
13m 8s (- 3m 17s) (8000 80%) 2.8367
14m 49s (- 1m 38s) (9000 90%) 2.8588
16m 31s (- 0m 0s) (10000 100%) 2.7943


In [181]:
evaluateRandomly(encoder1, decoder1)

> мы не продаем наш дом .
= we re not selling our house .
< we re all to . you . <EOS>

> он слегка навеселе .
= he s a bit tipsy .
< he is a to . . <EOS>

> он адвокат .
= he s a lawyer .
< he is a . . <EOS>

> мы готовы к любому развитию событии .
= we re ready for whatever might come .
< we re all to . you . <EOS>

> вы не сумасшедшая .
= you aren t crazy .
< you re a . . <EOS>

> мы новые владельцы .
= we re the new owners .
< we re all . . <EOS>

> она ученыи и поэт .
= she is a scholar and poet .
< she is a . . <EOS>

> я уверен что он не сделал ничего плохого .
= i am convinced that he did nothing wrong .
< i m sure you ll you . <EOS>

> боюсь завтра может поити дождь .
= i m afraid it may rain tomorrow .
< i m going to you you . . <EOS>

> я немного занят .
= i m kind of busy .
< i m going to . . <EOS>



#### 3. + RNN layer

The Encoder + RNN layer
-----------





In [191]:
class EncoderRNN_RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_RNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder + RNN layer
-----------




In [192]:
class DecoderRNN_RNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN_RNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [193]:
hidden_size = 256
encoder3 = EncoderRNN_RNN(input_lang.n_words, hidden_size).to(device)
decoder3 = DecoderRNN_RNN(hidden_size, output_lang.n_words).to(device)
print(encoder3, decoder3)
trainIters(encoder3, decoder3, 10000, print_every=1000)

EncoderRNN_RNN(
  (embedding): Embedding(9706, 256)
  (gru): GRU(256, 256)
) DecoderRNN_RNN(
  (embedding): Embedding(4150, 256)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=4150, bias=True)
  (softmax): LogSoftmax(dim=1)
)
1m 37s (- 14m 39s) (1000 10%) 3.6035
3m 17s (- 13m 8s) (2000 20%) 3.3654
4m 54s (- 11m 27s) (3000 30%) 3.2711
6m 31s (- 9m 47s) (4000 40%) 3.0094
8m 11s (- 8m 11s) (5000 50%) 2.9983
9m 51s (- 6m 34s) (6000 60%) 2.9260
11m 31s (- 4m 56s) (7000 70%) 2.8561
13m 10s (- 3m 17s) (8000 80%) 2.8228
14m 52s (- 1m 39s) (9000 90%) 2.8082
16m 35s (- 0m 0s) (10000 100%) 2.7640


In [194]:
evaluateRandomly(encoder3, decoder3)

> я почти закончил .
= i m almost finished .
< i m a . . <EOS>

> я тоже не голоден .
= i m not hungry either .
< i m not to of you . . <EOS>

> не уверена .
= i m not certain .
< i m not to . . . <EOS>

> я на тебя не сержусь .
= i m not angry with you .
< i m not to to tom . . <EOS>

> вы точно такои же как ваш отец .
= you re just like your father .
< you re a of you . . <EOS>

> на мне старая пижама тома .
= i m wearing tom s old pajamas .
< i m not to to tom . . <EOS>

> я не буду вас слушать .
= i m not going to listen to you .
< i m not to to tom . . <EOS>

> я свободен сеичас .
= i m free at the moment .
< i m a of . . <EOS>

> я пришел повидать тома .
= i m here to see tom .
< i m not to to tom . . . <EOS>

> он всегда приходит на встречи вовремя .
= he is always on time for an appointment .
< he is a of of . . <EOS>



#### 4. Заменим GRU layer на LSTM layer

The Encoder + LSTM layer
-----------





In [195]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        # self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded 
        # output, hidden = self.gru(output, hidden)
        hidden = (hidden, torch.zeros(1, 1, self.hidden_size))              
        output, (hidden, cell) = self.lstm(output, hidden)
        output, (hidden, cell) = self.lstm(output, (hidden, cell))
        
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device))

In [196]:
encoder4 = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
encoder4

EncoderLSTM(
  (embedding): Embedding(9706, 256)
  (lstm): LSTM(256, 256)
)

The Decoder + LSTM layer
-----------




In [197]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        
        hidden = (hidden, torch.zeros(1, 1, self.hidden_size))
        output, (hidden, cell)= self.lstm(output, hidden)
        output, (hidden, cell) = self.lstm(output, (hidden, cell))
       
        output = self.softmax(self.out(output[0]))        
        return output, hidden

    def initHidden(self):
      
        return (torch.zeros(1, 1, self.hidden_size, device=device))  

In [200]:
hidden_size = 256
encoder4 = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder4 = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

print (encoder4, decoder4)
trainIters(encoder4, decoder4, 10000, print_every=1000)


EncoderLSTM(
  (embedding): Embedding(9706, 256)
  (lstm): LSTM(256, 256)
) DecoderLSTM(
  (embedding): Embedding(4150, 256)
  (lstm): LSTM(256, 256)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=4150, bias=True)
  (softmax): LogSoftmax(dim=1)
)
2m 5s (- 18m 52s) (1000 10%) 3.7047
4m 36s (- 18m 24s) (2000 20%) 3.3428
7m 6s (- 16m 35s) (3000 30%) 3.2194
9m 21s (- 14m 1s) (4000 40%) 3.2795
11m 23s (- 11m 23s) (5000 50%) 3.1912
13m 24s (- 8m 56s) (6000 60%) 3.1174
15m 27s (- 6m 37s) (7000 70%) 3.1110
17m 29s (- 4m 22s) (8000 80%) 3.0528
19m 32s (- 2m 10s) (9000 90%) 3.1255
21m 35s (- 0m 0s) (10000 100%) 3.0939


In [201]:
evaluateRandomly(encoder4, decoder4)

> я не лгунья .
= i m no liar .
< i m going . . . . . . .

> ты такои трус .
= you re such a coward .
< i m going . . . . . . .

> ты идеален .
= you re perfect .
< i m going . . . . . . .

> я один из вас .
= i m one of you .
< i m going . . . . . . .

> они врачи .
= they re doctors .
< i m going . . . . . . .

> я озадачена .
= i m puzzled .
< i m going . . . . . . .

> мы не спим .
= we re awake .
< i m going . . . . . . .

> вы очень высокии .
= you re very tall .
< i m going . . . . . . .

> мне надоела рыба .
= i m sick of eating fish .
< i m going . . . . . . .

> я не спрашиваю тома .
= i m not asking tom .
< i m going . . . . . . .



#### Вывод: По качеству и скорости лучше отработала модель с ячейками GRU